## Cost Function jupyter notebook

In [47]:
import pandas as pd
import numpy as np
import math 
from decimal import ROUND_DOWN, ROUND_UP


In [48]:
import data
import input_data

In [49]:
lamination = data.lamination_data
swg = data.swg_data

In [50]:
Frequency = 47 # Hz
Temperature_rise_goal = 100 # degree Celcius
Output_power = 250 # watts
Efficiency = 95 # %
Input_voltage = 115 # volts
Ouptut_voltage = 115 # volts
Regulation = 5 # in a scale of 100
Bobbin_thickness = 1.5 
K_f = 4.44
K_u = 0.4
B_ac = 1.6
J = 250
insulation_thickness = 0.2 #mm

pi = np.pi #pi
a = 1.68 # coefficients for core loss
b = 1.86 # coefficients for core loss 

Rate_of_Cu = 950 # Rs / Kg
Rate_of_Fe = 250 # Rs / Kg

Resistivity_Cu =  1.68 * 10**-6 # ohm cm

In [51]:
def area_product(K_f, K_u, B_ac, J, Frequency, apparent_power):
    """Input Arguments:
        Output power watts
        Efficiency in scale of 100%
        K_f
        K_u
        B_ac -> Operational flux density
        J
        Frequency -> in Hz
        Apparent power in watts
    """
    Area_product = (apparent_power*(10**4))/(K_f * K_u * B_ac * J * Frequency)
    return Area_product

def InputCurrent(Apparent_power, Input_voltage, Efficiency):
    # Calculation of input current
    InputCurrent = Apparent_power / ( Input_voltage * Efficiency * 0.01) 
    return InputCurrent
    
def bare_area(input_current, current_density):
    '''
        arguments: 
            current [amps], 
            current density [amp/]
        return variables: Bare area in sqmm
    '''
    a_wp = input_current / current_density
    return a_wp * 100

def findSWG(Bare_area):
    '''
        Bare area in sqmm
        return variables: 
            required swg dataframe, 
            diameter of insulated wire, 
            bare area of selected swg
    '''
    swg_data = data.swg_data
    # swg_data = pd.read_csv('../DATA/EMD - Sheet1.csv') # select the swg the data 
    higer_data = swg_data[Bare_area < swg_data['Normal Conductor Area mm²']]
    required_swg_result = higer_data.iloc[(higer_data['Normal Conductor Area mm²'] - Bare_area).abs().argsort()[:1]]
    diameter_of_insulated_wire = required_swg_result['Medium Covering Max']
    A_wp = required_swg_result['Normal Conductor Area mm²'].max() / 100 # cm^2
    return required_swg_result, diameter_of_insulated_wire,  A_wp 

"""
Now functions of the task are created 
"""
def calculate_stack(area_product, current_lamination):
    '''
    current lamination is just the type of lamination in lamiantion
    data eg: SB1R, 17, 74 as string
    '''
    lamination_data = data.lamination_data
    selected_lamination = lamination_data[lamination_data['Type'] == current_lamination]
    stack_in_cm = area_product * 1000 / selected_lamination['K-ratio'].max()
    stack = stack_in_cm * 10 # mm
    return stack 

def rounding_stack_as_multiple_of_five(stack):
    # for approximating for stack 
    if stack < 5:
        stack = 5.0
    elif stack%5 == 0:
        stack = stack
    elif stack%5 <= 2.5:
        stack = stack - stack % 5
    elif stack%5 > 2.5:
        stack = stack - stack % 5 + 5
    return stack





In [53]:
lamination_data = data.lamination_data
for lamination in lamination_data['Type']:
    print(lamination)

SB1R
SB3R
17
74
12A
74
23
30
31
15
33
3
16
6
43
8
8B
8C
I180
I120
S14
S12
S15


In [ ]:
def Complete_Cost_function(Output_power, Efficiency, Frequency, K_f, K_u, B_ac, J,Output_voltage, Input_voltage):
   """
   This function derives from the user inputs like
   -> Frequeny, Input voltage, Output voltage, Regulation,
      K_f, K_u, B_ac, Current density, insulation thickness
      Rate of Cu, Rate of Iron

   Output consists of optimized cost with required parameters 
   for designing the transformer
   """

   # Adding data files into the funciton
   lamination_data = data.lamination_data
   swg_data = data.swg_data

   # Calculate the apparent power
   Apparent_power = Output_power * (1/(0.01*Efficiency) + 1)
   print('Apparent Power: ' + str(Apparent_power) + ' watts')

   # Calculate the area product
   Area_product = area_product(K_f, K_u, B_ac, J, Frequency, Apparent_power)
   print('Area product: ' + str(Area_product))

   # Calculate the input current
   Input_current = InputCurrent(Apparent_power=Apparent_power, Input_voltage= Input_voltage, Efficiency= Efficiency)
   print('Input current ' + str(Input_current) + ' [amps]')
   
   # Calculate Bare area of primary wire
   A_wp_in_sqmm = bare_area(Input_current, J) 
   print('Calculated Priamry Bare Area: ' + str(A_wp_in_sqmm)+ ' mm²')

   # Calculate swg for primary wire
   required_swg_primary, diameter_of_primary_wire_with_insulation, A_wp = findSWG(A_wp_in_sqmm)
   print('SWG: ' + str(required_swg_primary['SWG'].to_string(index=False)))
   print('Final Priamry Bare Area after selecting swg: ' + str(A_wp*100)+ ' mm²')
   print('Primary wire Diameter with enamel: ' + str(diameter_of_primary_wire_with_insulation.max()) + ' mm')

   # Find Secondary current
   Secondary_current = Output_power / Output_voltage

   # Calculate Bare Area for secondary wire in cm
   A_ws = bare_area(Secondary_current, J)
   print('Secondary Bare Area: ' + str(A_ws)+ ' mm²') 

   # Find SWG for secondary wire as well as Actual Bare are
   required_swg_secondary, diameter_of_wire_secondary_insulated , A_ws = findSWG(A_ws)
   print('SWG: ' + str(required_swg_secondary['SWG'].to_string(index=False)))
   print('Primary wire Diameter with enamel: ' + str(diameter_of_wire_secondary_insulated.max()) + ' mm')

   


   




# testing
if __name__ == '__main__':
   Complete_Cost_function(Output_power, Efficiency, Frequency, K_f, K_u, B_ac, J, Ouptut_voltage, Input_voltage)



Apparent Power: 513.1578947368421 watts
Area product: 153.6916255561272
Input current 4.69709743466217 [amps]
Calculated Priamry Bare Area: 1.8788389738648679 mm²
SWG: 16.0
Final Priamry Bare Area after selecting swg: 2.0765 mm²
Primary wire Diameter with enamel: 1.737 mm
Secondary Bare Area: 0.8695652173913043 mm²
SWG: 18.5
Primary wire Diameter with enamel: 1.216 mm
